<a href="https://colab.research.google.com/github/AlexKressner/Business_Intelligence/blob/main/ML_Fallstudie_Online_Barber_Shop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study "Online Barber Shop"
Sie finden unter `./Daten/Barber/` den Ausschnitt eines Datensatzes einer **Direct-to-Consumer (D2C) Brand**, die über einen Onlineshop **Rasierer** und **Rasierzubehör** verkauft. Für das Unternehmen ist es enorm wichtig, dass es die voraussichtlichen **Absätze der nächsten 21 Tage** kennt, da es Waren im Voraus bei seinen Lieferanten bestellen muss. Je genauer die Absatzprognose, desto weniger Stockouts bzw. unnötige Bestände hat das Unternehmen und desto besser die Marge.

Bisher ist es dem Unternehmen nicht gelungen, gute Prognosen zu erstellen. Um dies zu ändern, werden Sie nun beauftragt, eine Prognose mithilfe von ML-Algorithmen anzufertigen. Dazu stellt Ihnen das Unternehmen einen ersten Datensatz aus seinem Shopsystem zur Verfügung. In diesem sind **3 Produkte** enthalten (Alaunstein, Bartbalm und ein Rasierer) und Sie sollen für diese jeweils eine Absatzprogonse erstellen. Bitte orientieren Sie sich bei der Bearbeitung der Fallstudie an der in diesem Notebook vorgeschlagenen Struktur und greifen Sie auf die Notebooks zurück, die wir bereits in der Vorlesung gemeinsam bearbeitet haben.

Die Struktur Ihres Notebooks sollte dabei wie folgt aussehen und u.a. die folgenden Fragen beantworten:

1. **Daten laden & aufbereiten**. Wichtig: Nachdem Sie die Daten geladen haben, müssen Sie diese zunächst auf Tage und je Produkt aggregieren.
2. **Datenexploration**, d.h. verschaffen Sie sich einen Überblick zum Datensatz. Sie sollten folgende Fragen beantworten:
  - Wie viele Absätze wurden über welchen Zeitraum je Produkt dokumentiert?
  - Was ist der Gesamtumsatz je Produkt?
  - Wie sind die Produktverkäufe im Zeitverlauf? Visualisieren Sie beispielweise die monatlichen/wöchentlichen Verkaufsmengen. Dies ist wichtig, um Trends oder Saisonalität in Zeitreihen zu erkennen.
  - Wie sind die durchschnittlichen Absätze pro Wochentag?
3. **Feature Engineering**
  - Entwickeln Sie basirend auf Ihrer Datenexploration nützliche Features für ein Vorhersagemodell.
  - Stellen Sie sich die Frage, mit welchen Features Sie Saisonalitäten abbilden können. Gibt es Monate in den die Absätze deutlich höher sind? Falls ja, sollten Sie ein Features einführen, das den Monat (Jan, Feb, ..., Dez) abbildet!
  - Bilden Sie autoregressive Features, z.B. könnte es Sinn ergeben zur Progonse des Absatzes an einem Tag (z.B. 15.03.) den Absatz des Vormonats zum gleichen Datum (15.02.) heranzuziehen.
  - Können Sie aus den zur Verfügung stehenden Daten den Stückpreis als Feature ableiten?
4. **Prognosemodell trainieren und bewerten**
  - Trainieren Sie ein globales Prognosemodell auf allen Daten und bewerten Sie dieses.
  - Trainieren Sie drei lokale Prognosemodelle (für Alaunstein, Bartbalm & Rasierer) und überprüfen Sie, ob die Prognosegüte verbessert wird.


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv(r'Daten\Barber\barber_shopify_data.csv', sep=';', parse_dates=['Datum'], decimal=',')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.head()

In [ ]:
df.columns = df.columns.str.replace('Versand Land', 'Versand_Land')

for col in df.columns:
    df.rename(columns={col: col.lower()}, inplace=True)
df.head()

In [ ]:
def preprocess_data(df):

    min_date = min(pd.to_datetime(df["datum"]))
    max_date = max(pd.to_datetime(df["datum"]))

    dates = (
        pd.date_range(start=min_date, end=max_date, name="datum")
        .to_frame()
        .reset_index(drop=True)
    )
    dates = (
        pd.merge(dates, df["produkt"], how="cross")
        .drop_duplicates()
        .reset_index(drop=True)
    )

    return pd.merge(dates, df, how="left", on=["datum", "produkt"]).fillna(0)


df_agg = df.copy().groupby(["datum", "produkt"]).agg({"nettomenge": "sum", "bruttoumsatz": "sum"}).reset_index()


In [ ]:
def get_agg_for_col(df: pd.DataFrame, group_col: list[str] | str, agg_col: str, agg_func: str = 'sum', sorted: bool = False, ascending: bool = False) -> pd.DataFrame:
    if isinstance(group_col, str):
        group_col = [group_col]
    if agg_func == 'sum':
        agg_result = df.groupby(group_col)[agg_col].sum()
    elif agg_func == 'mean':
        agg_result = df.groupby(group_col)[agg_col].mean()
    elif agg_func == 'count':
        agg_result = df.groupby(group_col)[agg_col].count()
    elif agg_func == 'min':
        agg_result = df.groupby(group_col)[agg_col].min()
    elif agg_func == 'max':
        agg_result = df.groupby(group_col)[agg_col].max()
    else:
        raise ValueError(f"Invalid aggregation function: {agg_func}")
    
    agg_result = agg_result.reset_index()
    
    if sorted:
        agg_result = agg_result.sort_values(by=agg_col, ascending=ascending)
    
    return agg_result


In [ ]:
df_agg.head()

In [ ]:
agg_cols = ['nettomenge', 'bruttoumsatz']
for col in agg_cols:
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=get_agg_for_col(df, ['datum', 'produkt'], col ), x='datum', y=col, hue='produkt')
    plt.show()

In [ ]:
for col in ['nettomenge', 'bruttoumsatz']:
    plt.figure(figsize=(10, 6))
    plt.title(f'Summe {col} pro Produkt')
    sns.barplot(data=get_agg_for_col(df, 'produkt', 'nettomenge', sorted=True), x='produkt', y='nettomenge')
    plt.show()


In [ ]:
def add_datetime_features(df: pd.DataFrame) -> pd.DataFrame:
    df['year'] = df['datum'].dt.year
    df['month'] = df['datum'].dt.month
    df['day'] = df['datum'].dt.day
    df['week'] = df['datum'].dt.isocalendar().week
    df['weekday'] = df['datum'].dt.weekday
    df['day_of_year'] = df['datum'].dt.dayofyear
    df['quarter'] = df['datum'].dt.quarter
    return df

In [ ]:
df_agg.head()

In [ ]:
df_agg = add_datetime_features(df_agg)
df_agg['day_name'] = df_agg['datum'].dt.day_name()

plt.figure(figsize=(10, 6))
sns.barplot(data=get_agg_for_col(df_agg, ['day_name', 'produkt'], 'nettomenge', agg_func='mean'), x='day_name', y='nettomenge', hue='produkt')
plt.show()

In [ ]:
df_agg.sort_values(by="datum", inplace=True, ascending=True)

df_agg["sales_lag1d"] = df_agg.groupby("produkt")["nettomenge"].transform(
    lambda x: x.shift(21)
)
df_agg["3d_rolling_demand"] = df_agg.groupby("produkt")["nettomenge"].transform(
    lambda x: x.shift(21).rolling(window=3).mean()
)
df_agg.tail(10)

In [ ]:
def create_rolling_transform(df: pd.DataFrame, groupby_cols: list[str], column: str, shift_val: int, window_size: int) -> pd.DataFrame:
    df[column + f"_lag{shift_val}d"] = df.groupby(groupby_cols)[column].transform(lambda x: x.shift(shift_val))
    df[column + f"_{window_size}d_rolling_demand"] = df.groupby(groupby_cols)[column].transform(lambda x: x.shift(shift_val).rolling(window=window_size).mean())
    return df

In [ ]:
df_agg["item_value"] = df_agg["bruttoumsatz"] / df_agg["nettomenge"]
df_agg.dropna(inplace=True)
df_agg.drop(columns="day_name", inplace=True)

In [ ]:
df_agg.set_index("datum", inplace=True)

## Model global

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    mean_absolute_percentage_error,
)
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from xgboost import XGBRegressor

import joblib
import os


WORKDIR = os.getcwd()
MODELDIR = os.path.join(WORKDIR, "models")
data = os.path.join(WORKDIR, "Daten", "Barber", "barber_shopify_data.csv")

if not os.path.exists(MODELDIR):
    os.makedirs(MODELDIR)


# preprocessing functions
def add_datetime_features(df: pd.DataFrame) -> pd.DataFrame:
    df["year"] = df["datum"].dt.year
    df["month"] = df["datum"].dt.month
    df["day"] = df["datum"].dt.day
    df["week"] = df["datum"].dt.isocalendar().week.astype(int)
    df["weekday"] = df["datum"].dt.weekday
    df["day_of_year"] = df["datum"].dt.dayofyear
    df["quarter"] = df["datum"].dt.quarter
    return df


def create_rolling_transform(
    df: pd.DataFrame,
    groupby_cols: list[str],
    column: str,
    shift_val: int,
    window_size: int,
) -> pd.DataFrame:
    df.sort_values(by="datum", inplace=True, ascending=True)
    df[f"{column}_lag{shift_val}d"] = df.groupby(groupby_cols)[column].transform(
        lambda x: x.shift(shift_val)
    )

    df[f"{column}_{window_size}d_rolling"] = df.groupby(groupby_cols)[column].transform(
        lambda x: x.shift(shift_val).rolling(window=window_size).mean()
    )
    return df


def calc_avg_item_value(df: pd.DataFrame) -> pd.DataFrame:
    df["item_value"] = df["bruttoumsatz"] / df["nettomenge"]
    return df


# load data
df = pd.read_csv(data, sep=";", parse_dates=["Datum"], decimal=",")


df.columns = df.columns.str.replace("Versand Land", "Versand_Land")
for col in df.columns:
    df.rename(columns={col: col.lower()}, inplace=True)

# preprocess data
df_agg = (
    df.copy()
    .groupby(["datum", "produkt"])
    .agg({"nettomenge": "sum", "bruttoumsatz": "sum"})
    .reset_index()
)

# preprocess_data(df_agg)
add_datetime_features(df_agg)
calc_avg_item_value(df_agg)

rolling_kwargs = {}
for shift_val in range(30, 365, 7):
    for window_size in [3, 7, 30, 90, 180, 365]:
        rolling_kwargs[f"{shift_val}_{window_size}"] = {
            "groupby_cols": ["produkt"],
            "column": "nettomenge",
            "shift_val": shift_val,
            "window_size": window_size,
        }
for key, value in rolling_kwargs.items():
    create_rolling_transform(df_agg, **value)

df_agg.set_index("datum", inplace=True)
df_agg.drop(columns="bruttoumsatz", inplace=True)


# create holdout set last 30 days
holdout_split = "2022-06-01"
df_agg = df_agg[df_agg.index < "2022-07-01"]
holdout_data = df_agg[df_agg.index >= holdout_split]
df_agg_train_test = df_agg[df_agg.index < holdout_split]
df_agg.head()
models = {}

# Define the parameter grid for grid search
param_grid = {
    "n_estimators": [100, 500, 1000],
    "learning_rate": [0.1, 0.01, 0.001],
    "max_depth": [3, 5, 7],
}

# Perform time series cross-validation and grid search for each product
for product in df_agg_train_test["produkt"].unique():
    df_global_train_test = (
        df_agg_train_test[df_agg_train_test["produkt"] == product]
        .drop(columns="produkt")
        .copy()
    )

    X = df_global_train_test.drop(columns=["nettomenge"])
    y = df_global_train_test["nettomenge"]

    model = XGBRegressor(random_state=42, objective="reg:squarederror", n_jobs=-1)

    tscv = TimeSeriesSplit(n_splits=5)

    grid_search = GridSearchCV(
        model, param_grid, cv=tscv, scoring="neg_mean_squared_error"
    )
    grid_search.fit(X, y)

    best_model = grid_search.best_estimator_

    model_name = f"{product}_model"
    file_name = f"{model_name}.joblib"

    grid_search = GridSearchCV(
        model, param_grid, cv=tscv, scoring="neg_mean_squared_error"
    )
    grid_search.fit(X, y)

    best_model = grid_search.best_estimator_

    model_name = f"{product}_model"
    file_name = f"models/{model_name}.joblib"

    joblib.dump(best_model, file_name)

    models[product] = best_model

In [ ]:
forecast_df = pd.DataFrame()
for product in models:
    model_path = f"models/{product}_model.joblib"  
    model = joblib.load(model_path)
    
    holdout_data_product = holdout_data[holdout_data['produkt'] == product].drop(columns='produkt').copy()
    X_holdout = holdout_data_product.drop(columns=["nettomenge"])
    y_holdout = holdout_data_product["nettomenge"].reset_index()

    y_forecast_new = model.predict(X_holdout)
    y_forecast_new = pd.DataFrame(y_forecast_new, index=y_holdout['datum'], columns=["forecast"]).reset_index()
    y_forecast_new["produkt"] = product
    forecast_df_temp= pd.merge(y_holdout, y_forecast_new, how='inner', on='datum')
    forecast_df = pd.concat([forecast_df, forecast_df_temp] , axis=0)

    mse = mean_squared_error(y_holdout["nettomenge"], y_forecast_new["forecast"])
    mae = mean_absolute_error(y_holdout["nettomenge"], y_forecast_new["forecast"])
    mape = mean_absolute_percentage_error(y_holdout["nettomenge"], y_forecast_new["forecast"])


    print(f"Errors for {product}:")
    print(f"Mean Squared Error: {mse}")
    print(f"Mean Absolute Error: {mae}")
    print(f"Mean Absolute Percentage Error: {mape}\n")

forecast_df_total = forecast_df.groupby(['datum']).agg({'nettomenge': 'sum', 'forecast': 'sum'}).reset_index()
mse = mean_squared_error(forecast_df_total["nettomenge"], forecast_df_total["forecast"])
mae = mean_absolute_error(forecast_df_total["nettomenge"], forecast_df_total["forecast"])
mape = mean_absolute_percentage_error(forecast_df_total["nettomenge"], forecast_df_total["forecast"])

print(f"{'_'*20}\n")
print("Errors for the total forecast:")
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"Mean Absolute Percentage Error: {mape}\n")

In [ ]:
# Plot the forecast
forecast_df_total = forecast_df.groupby(['datum']).agg({'nettomenge': 'sum', 'forecast': 'sum'}).reset_index()
plt.figure(figsize=(10, 6))
sns.lineplot(data=forecast_df_total, x='datum', y='nettomenge', label='actual')
sns.lineplot(data=forecast_df_total, x='datum', y='forecast', label='forecast')
plt.title('Forecast vs Actual')
plt.show()
